### Baixa a lista de processos TJSP com base no nome da parte


In [1]:
# Importando as Bibliotecas
import pandas as pd
import requests 
from bs4 import BeautifulSoup

In [115]:
busca = ""
busca = busca.replace(' ','+')

In [116]:
foros = [1, 2, 3, 4, 5, 6,7, 8, 9, 10, 11, 12,13, 14, 15, 16, 17, 18,19, 20, 21, 22, 24, 25,28, 30, 32, 35, 37, 38,40, 42, 45, 47, 48, 50,52, 53, 58, 59, 60, 62,
63, 66, 67, 68, 69, 70,71, 72, 73, 75, 76, 77,79, 80, 81, 82, 83, 84,85, 86, 87, 88, 89, 90,91, 93, 94, 95, 97, 99,100, 101, 102, 103, 104, 106,108, 111, 114, 115, 116, 118,
120, 123, 125, 126, 127, 128,129, 132, 136, 137, 140, 142,144, 145, 146, 150, 152, 153,156, 157, 159, 160, 161, 165,168, 169, 172, 176, 177, 180,185, 187, 189, 191, 196, 197,
198, 200, 201, 204, 205, 210,213, 218, 219, 220, 222, 223,224, 229, 233, 236, 238, 240,242, 244, 246, 247, 248, 252,257, 262, 263, 264, 266, 268,269, 270, 271, 272, 274, 275,
278, 279, 280, 281, 282, 283,286, 288, 291, 292, 294, 296,297, 299, 300, 301, 302, 306,309, 311, 312, 315, 318, 319,320, 322, 323, 326, 333, 334,337, 338, 341, 344, 346, 347,
348, 352, 355, 356, 357, 358,360, 361, 362, 363, 366, 368,369, 370, 372, 374, 382, 383,390, 394, 396, 397, 400, 404,405, 407, 408, 411, 412, 414,415, 416, 417, 418, 420, 424,
426, 428, 430, 431, 434, 435,438, 439, 441, 443, 444, 445,447, 449, 450, 451, 452, 453,456, 457, 458, 459, 462, 464,466, 470, 471, 472, 474, 477,480, 481, 482, 483, 484, 486,
488, 491, 493, 495, 498, 505,506, 510, 511, 512, 515, 516,523, 526, 531, 533, 534, 538,539, 541, 543, 547, 549, 553,554, 562, 563, 564, 565, 566,568, 572, 575, 576, 577, 579,
581, 582, 584, 586, 587, 588,589, 590, 595, 596, 597, 601,602, 604, 606, 607, 609, 614,615, 619, 620, 624, 625, 627,629, 634, 637, 638, 642, 646,648, 650, 651, 653, 654, 655,
659, 660, 663, 664, 665, 666,667, 668, 669, 670, 671, 672,673, 674, 675, 676, 677, 678,679, 680, 681, 682, 683, 684,685, 686, 687, 688, 689, 690,691, 692, 693, 694, 695, 696,
697, 698, 699, 700, 701, 702,703, 704, 705, 706, 707, 708,709, 710, 990]

In [117]:
#foros = [266]
dados = []
procs_avulsos = []

In [ ]:
for foro in foros: 
    print("Pesquisando foro " + str(foro)) 
    url = "https://esaj.tjsp.jus.br/cpopg/search.do?conversationId=&cbPesquisa=NMPARTE&dadosConsulta.valorConsulta=" + busca + "&cdForo=" + str(foro)
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    mensagem = soup.find('td',attrs={'id':'mensagemRetorno'})
    proc_unico = soup.find('span',attrs={'id':'numeroProcesso'})
    
    # Aqui se nao encontrou nenhuma infomração, retorna a mensgagem 
    if mensagem:
        print(mensagem.find('li').text)
    
    # Caso so tenha um processo na vara, ele já abre na tela. Pega as infos deste cara. 
    elif proc_unico: 

        print(proc_unico.text.strip())
        nro_processo = proc_unico.text.strip()
        link_processo = ''
        nome_parte = ''
        classe_processo = soup.find('span',attrs={'id':'classeProcesso'}).text.strip()
        assunto_processo = soup.find('span',attrs={'id':'assuntoProcesso'}).text.strip()
        data_local_distribuicao = soup.find('div',attrs={'id':'dataHoraDistribuicaoProcesso'}).text.strip()

        linha = [nro_processo,link_processo,nome_parte,classe_processo, assunto_processo,data_local_distribuicao]

        if linha: 
            dados.append(linha)

    # Caso contrário, le todas as páginas
    else: 
        qtde_proc = soup.find("span",attrs={'id':'contadorDeProcessos'}).text.strip()
        qtde_proc = ''.join(c if c.isdigit() else '' for c in qtde_proc)

        paginas = round(int(qtde_proc)/25)

        for pag in range(1,paginas): 
            print("Pesquisando foro " + str(foro) + " pagina " + str(pag)) 
            url_pag = "https://esaj.tjsp.jus.br/cpopg/trocarPagina.do?paginaConsulta=" + str(pag) + "&conversationId=&cbPesquisa=NMPARTE&dadosConsulta.valorConsulta=" + busca + "&cdForo=" + str(foro)
            html = requests.get(url_pag).content
            soup = BeautifulSoup(html, 'html.parser')
            tabela = soup.find("div",attrs={'id':'listagemDeProcessos'})
        
            processos = tabela.find_all('div',attrs={'class':'row unj-ai-c home__lista-de-processos'})

            for proc in processos:
                nro_processo = proc.find('a',attrs={'class':'linkProcesso'}).text.strip()
                link_processo = proc.find('a',attrs={'class':'linkProcesso'}).attrs['href'].strip()
            
                try: 
                    nome_parte = proc.find('div',attrs={'class':'unj-base-alt nomeParte'}).text.strip()
                except:
                    pass
            
                classe_processo = proc.find('div',attrs={'class':'classeProcesso'}).text.strip()
                assunto_processo = proc.find('div',attrs={'class':'assuntoPrincipalProcesso'}).text.strip()
                data_local_distribuicao = proc.find('div',attrs={'class':'dataLocalDistribuicaoProcesso'}).text.strip()
        
                linha = [nro_processo,link_processo,nome_parte,classe_processo, assunto_processo,data_local_distribuicao]

                if linha: 
                    dados.append(linha)
    

In [ ]:
df_processos = pd.DataFrame(dados,columns=["nro_processo","link_processo","nome_parte","classe_processo","assunto_processo","data_local_distribuicao"])

df_processos

In [120]:
df_processos.to_csv('tjsp_processos_xxxxx.csv',sep=";")